In [16]:
import multiprocessing
from pathlib import Path
import pandas as pd

In [17]:
all_data = Path('../testdata/').glob('*.vcf')

In [18]:
all_path = []
for d in all_data:
    all_path.append(d)
    
    
    
controls = [x.name for x in all_path[0:10] ]
patients = [x.name for x in all_path[10:]]

In [4]:
# function to get array with chromsome, position, ref, alt, in single .vcf file
def getindex(file):
    array = []
    with file.open() as f:
        for line in f:
            if line.startswith(current_chromosome):
                try:
                    block = line.split('\t')
                    chromosome = block[0]
                    position = block[1]
                    ref = block[3]
                    alt = block[4]
                    PK = chromosome + ':' + position + ':' + ref + ':' + alt

                    array.append(PK)
                except:
                    print(line + '---  error')
                
    return(array)

In [5]:
# function to append array from every vcf to unique set.

def addtoset(result):
    for res in result:
        for r in res:
            shareset.add(r)

In [6]:
## map type
def getGenotype(single_vcf_file):
    vcf = single_vcf_file.open()

    PK_sample = dict()  # dict(vcf_per_line:index)
    Sample = []
    line_count = 0

    for num, line in enumerate(vcf):
        if line.startswith(current_chromosome):
            block = line.split('\t')
            chromosome = block[0]
            position = block[1]
            ref = block[3]
            alt = block[4]
            PK = chromosome + ":" + position + ":" +ref + ":" + alt
            PK_sample[PK] = line_count
            line_count += 1
            Sample.append(line)

    genotype = ['none']*len(shareset_list)
    for i in range(0, len(shareset_list)):
        per_unique_loc = shareset_list[i]

        # get the vcf line of sample, and transfer the GT label.
        if per_unique_loc in PK_sample.keys():
            index = PK_sample[per_unique_loc]
            block = Sample[index].split('\t')
            ref = block[3]
            alt = block[4].split(',')
            if alt != '<CNV>':
                table = [ref] + [a for a in alt]
                GT = block[9].split(':')[0]
                genotype_row = []
                for geno_num in GT.split('/'):
                    genotype_row.append(table[int(geno_num)])

                # print(genotype_row, i)
                genotype[i] = ''.join(genotype_row)
        else:
            ref = per_unique_loc.split(':')[2]
            genotype[i] = ""+ref+ref

            
    
    genotype.append(single_vcf_file.name)

    return genotype

In [ ]:
# get size 
from sys import getsizeof
getsizeof(shareset)/1024/1024

In [7]:

def addtolist(l):
    for res in l:
        GT[res[-1]] = res[:-1]


In [8]:
from scipy import stats

def calfisher(row):
    
    index = row[0].split(":")
    
    chrom = index[0][3:]
    pos = index[1]
    
    ref = index[2]
    ref = ref+ref

    Con_AA = 0
    Con_AB = 0

    for con in controls:
        if row[1][con] == ref:
            Con_AA += 1
        else:
            Con_AB += 1

    Pat_AA = 0
    Pat_AB = 0
    for pat in patients:
        if row[1][pat] == ref:
            Pat_AA += 1
        else:
            Pat_AB += 1

    oddsratio, pvalue = stats.fisher_exact([[Con_AA, Pat_AA],[Con_AB,Pat_AB]])

    return [chrom, pos, pvalue]

In [9]:
def addtoresult(l):
    for res in l:
        result.append(res)

In [21]:
chr_set = ['chr' + str(x) for x in range(1,23) ] + ['chrX','chrY','chrM']

for current_chromosome in chr_set:
    shareset = set()
    p = multiprocessing.Pool(4)

    p.map_async(getindex, all_path, callback=addtoset)
    p.close()
    p.join()

    df = pd.DataFrame(index=shareset)

    shareset_list = []
    for line in shareset:
        shareset_list.append(line)
    shareset_list = sorted(shareset_list)

    GT = dict()

    p = multiprocessing.Pool(4)
    p.map_async(getGenotype, all_path, callback=addtolist)
    p.close()
    p.join()

    df = pd.DataFrame(GT)
    df.index = shareset_list
    
    result = list()

    p = multiprocessing.Pool(4)
    p.map_async(calfisher, df.iterrows(), callback=addtoresult)
    p.close()
    p.join()

    pd.DataFrame(result, columns=['CHR','BP','P']).to_csv('./' + current_chromosome +'.csv')

In [78]:
calfisher(next(df.iterrows()))

['1', '10000', 1.0]

In [13]:
current_chromosome

'chr1'

In [14]:
GT

{}

In [19]:
all_path

[PosixPath('../testdata/small_2013NGS002B1_TSVC.vcf'),
 PosixPath('../testdata/small_2013NGS005E1_TSVC.vcf'),
 PosixPath('../testdata/small_2013NGS008H1_TSVC.vcf'),
 PosixPath('../testdata/small_2013NGS010B2_TSVC.vcf'),
 PosixPath('../testdata/small_2013NGS018B3_TSVC.vcf'),
 PosixPath('../testdata/small_2013NGS020D3_TSVC.vcf'),
 PosixPath('../testdata/small_2013NGS024H3_TSVC.vcf'),
 PosixPath('../testdata/small_2013NGS035C5_TSVC.vcf'),
 PosixPath('../testdata/small_2013NGS036D5_TSVC.vcf'),
 PosixPath('../testdata/small_2013NGS042B6_TSVC.vcf'),
 PosixPath('../testdata/small_2013NGS056H7_TSVC.vcf'),
 PosixPath('../testdata/small_2013NGS059C8_TSVC.vcf'),
 PosixPath('../testdata/small_2013NGS068D9_TSVC.vcf'),
 PosixPath('../testdata/small_2013NGS072H9_TSVC.vcf'),
 PosixPath('../testdata/small_2013NGS078F10_TSVC.vcf'),
 PosixPath('../testdata/small_2013NGS082B11_TSVC.vcf'),
 PosixPath('../testdata/small_2013NGS088H11_TSVC.vcf'),
 PosixPath('../testdata/small_2013NGS094F12_TSVC.vcf'),
 Posix